Importing Libraries

In [3]:
!pip install optuna==3.1.0 --quiet numpy==1.22.4 neptune-client==0.16.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.7/444.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.6 MB/s eta 0:00:00


In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from  sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import optuna
from functools import partial
import neptune.new as neptune
import plotly.express as px


Initialising Neptune Run


In [5]:
run = neptune.init_run(api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3Njk5ODEwYy0wM2NmLTRlMDktYTA3NC00ZWFiZjVmZDMyNjIifQ==", project="d-mech.guy001/example-project-tensorflow-keras")


https://app.neptune.ai/d-mech.guy001/example-project-tensorflow-keras/e/TFKERAS-17
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


Loading the dataset

In [6]:
titanic=pd.read_csv("/content/trainSpaaceshipTitanic.csv")

titanic.shape

(8693, 14)

In [7]:
titanic = titanic.drop(["PassengerId", "Name","HomePlanet" , "Destination","Cabin"], axis=1)

titanic.head(20)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True
5,False,44.0,False,0.0,483.0,0.0,291.0,0.0,True
6,False,26.0,False,42.0,1539.0,3.0,0.0,0.0,True
7,True,28.0,False,0.0,0.0,0.0,0.0,NaN,True
8,False,35.0,False,0.0,785.0,17.0,216.0,0.0,True
9,True,14.0,False,0.0,0.0,0.0,0.0,0.0,True


In [8]:
titanic['Age']=titanic['Age'].fillna(titanic['Age'].mean(),inplace=False)
titanic['RoomService']=titanic['RoomService'].fillna(titanic['RoomService'].mean(),inplace=False)
titanic['FoodCourt']=titanic['FoodCourt'].fillna(titanic['FoodCourt'].mode()[0],inplace=False)
titanic['ShoppingMall']=titanic['ShoppingMall'].fillna(titanic['ShoppingMall'].mode()[0],inplace=False)
titanic['Spa']=titanic['Spa'].fillna(titanic['Spa'].mean(),inplace=False)
titanic['VRDeck']=titanic['VRDeck'].fillna(titanic['VRDeck'].mean(),inplace=False)


In [9]:
titanic.dropna(inplace=True)

In [10]:
titanic.head(20)


,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,False,39.0,False,0.0,0.0,0.0,0.0,0.000000,False
1,False,24.0,False,109.0,9.0,25.0,549.0,44.000000,True
2,False,58.0,True,43.0,3576.0,0.0,6715.0,49.000000,False
3,False,33.0,False,0.0,1283.0,371.0,3329.0,193.000000,False
4,False,16.0,False,303.0,70.0,151.0,565.0,2.000000,True
5,False,44.0,False,0.0,483.0,0.0,291.0,0.000000,True
6,False,26.0,False,42.0,1539.0,3.0,0.0,0.000000,True
7,True,28.0,False,0.0,0.0,0.0,0.0,304.854791,True
8,False,35.0,False,0.0,785.0,17.0,216.0,0.000000,True
9,True,14.0,False,0.0,0.0,0.0,0.0,0.000000,True


In [11]:
titanic.shape

(8280, 9)

Splitting into features and labels

In [12]:
X= titanic.drop("Transported",axis=1)

y=titanic["Transported"]

Splitting into training and testing data

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42, stratify=y)

Defining the preprocessing steps

In [14]:
numerical_features = ['Age', 'RoomService',	'FoodCourt',	'ShoppingMall',	'Spa',	'VRDeck']
categorical_features=['CryoSleep','VIP']

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
]
                        )
cat_pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy = 'most_frequent')),
    ('encoder',OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num',num_pipeline,numerical_features),
        ('cat', cat_pipeline, categorical_features)
    ],
    remainder = 'passthrough'
)

Creating the ML model

In [15]:
model = RandomForestClassifier(random_state = 42)

Building the pipeline

In [16]:
pipeline =Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

Performing Cross-Validation using StratifiedKFold

In [17]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='accuracy')

run["cross_val_accuracy_scores"]= cv_scores

run["mean_cross_val_accuracy_scores"]= np.mean(cv_scores)

<ipython-input-17-efd633d92db3>:5: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  run["cross_val_accuracy_scores"]= cv_scores


In [18]:
titanic.describe


/usr/local/lib/python3.10/dist-packages/IPython/lib/pretty.py:700: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)


<bound method NDFrame.describe of      CryoSleep   Age    VIP  RoomService  FoodCourt  ShoppingMall     Spa  \
0        False  39.0  False          0.0        0.0           0.0     0.0   
1        False  24.0  False        109.0        9.0          25.0   549.0   
2        False  58.0   True         43.0     3576.0           0.0  6715.0   
3        False  33.0  False          0.0     1283.0         371.0  3329.0   
4        False  16.0  False        303.0       70.0         151.0   565.0   
...        ...   ...    ...          ...        ...           ...     ...   
8688     False  41.0   True          0.0     6819.0           0.0  1643.0   
8689      True  18.0  False          0.0        0.0           0.0     0.0   
8690     False  26.0  False          0.0        0.0        1872.0     1.0   
8691     False  32.0  False          0.0     1049.0           0.0   353.0   
8692     False  44.0  False        126.0     4688.0           0.0     0.0   

      VRDeck  Transported  
0        0.0 

Training the pipeline

In [19]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['CryoSleep', 'VIP'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

Evaluating the pipeline

In [20]:
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

run["accuracy"] = accuracy
run["precision"] = precision
run["recall"] = recall
run["f1"] = f1

Defining the hyperparameter search space using Optuna

In [21]:
def objective(x_train, y_train, pipeline,cv, trial: optuna.trial):
  params = {
      "classifier__n_estimators": trial.suggest_int('classifier__n_estimators',10,200),
      "classifier__max_depth": trial.suggest_int('classifier__max_depth',10,50),
      'classifier__min_samples_split': trial.suggest_int('classifier__min_samples_split',2,10),
      'classifier__min_samples_leaf': trial.suggest_int('classifier__min_samples_leaf',1,5),
      'classifier__max_features': trial.suggest_categorical('classifier__max_features',['auto','sqrt'])
  }

  pipeline.set_params(**params)

  scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
  mean_score = np.mean(scores)

  return mean_score
  print(mean_score)

Hyperparameter tuning by optuna

In [22]:
study = optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler(seed=42))

study.optimize(partial(objective, X_train, y_train, pipeline, cv),n_trials=50, timeout=None, gc_after_trial=True)

[I 2023-06-14 14:29:32,610] A new study created in memory with name: no-name-85cff3ec-a882-4ab6-8a38-82cb24a9cc56
[I 2023-06-14 14:29:35,875] Trial 0 finished with value: 0.7914653784219001 and parameters: {'classifier__n_estimators': 81, 'classifier__max_depth': 48, 'classifier__min_samples_split': 8, 'classifier__min_samples_leaf': 3, 'classifier__max_features': 'auto'}. Best is trial 0 with value: 0.7914653784219001.
[I 2023-06-14 14:29:36,519] Trial 1 finished with value: 0.7909822866344605 and parameters: {'classifier__n_estimators': 21, 'classifier__max_depth': 45, 'classifier__min_samples_split': 7, 'classifier__min_samples_leaf': 4, 'classifier__max_features': 'sqrt'}. Best is trial 0 with value: 0.7914653784219001.
[I 2023-06-14 14:29:40,362] Trial 2 finished with value: 0.7850241545893719 and parameters: {'classifier__n_estimators': 168, 'classifier__max_depth': 18, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 1, 'classifier__max_features': 'sqrt'}. Bes

setting best params and training the pipeline

In [23]:
pipeline.set_params(**study.best_trial.params)

pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['CryoSleep', 'VIP'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=10, max_features='auto',
                                        min_samples_leaf=5, min_samples_split=8,
                                        n_estimators=63, random_state=42))])

Evaluating for the best model

In [24]:
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test,y_pred)
f1=f1_score(y_test, y_pred)

run["accuracy_tuned"] = accuracy
run["precision_tuned"] = precision
run["recall_tuned"]  = recall
run["f1_tuned"] = f1

Logging hyperparameters , best trial parameters and best Neptune score

In [25]:
run['parameters'] = study.best_trial.params
run['best_trial'] = study.best_trial.number
run['best_score'] = study.best_value

Log the classification report and confusion matrix

In [30]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = pipeline.predict(X_test)

#Log classification report
report = classification_report(y_test, y_pred,output_dict =True)
for label,metrics in report.items():
  if isinstance(metrics,dict):
    for metrics,value in metrics.items():
      run[f"classification_report/{label}"]=metrics

#Log confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat_plot = px.imshow(conf_mat, labels=dict(x="Predict", y="Target"),x=[x+1 for x in range(len(conf_mat[0]))],y=[x+1 for x in range(len(conf_mat[0]))])

run['confusion_matrix'].upload(neptune.types.File.as_html(conf_mat_plot))




Log the pipeline as a pickle file.

In [31]:
import joblib

joblib.dump(pipeline, 'optimized_pipeline.pkl' )
run['optimized_pipeline'].upload(neptune.types.File.as_pickle('optimized_pipeline.pkl'))


stopping the neptune run

In [32]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/d-mech.guy001/example-project-tensorflow-keras/e/TFKERAS-17
